In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd 'drive/My Drive/'

/content/drive/My Drive


In [3]:
ls 'Selected'

train_color/  train_label/


In [4]:
!pip install segmentation_models

In [5]:
#Importing necessary libraries
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"  #- Uncomment this to run tensorflow on CPU
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline
from glob import glob

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import segmentation_models as sm

Segmentation Models: using `keras` framework.


Using TensorFlow backend.


In [6]:
IMG_HEIGHT = 256
IMG_WIDTH = 256

IMG_DIR  = os.path.join('.', 'Selected', 'train_color')
MASK_DIR = os.path.join('.', 'Selected', 'train_label')

BACKBONE = 'seresnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

INPUT_SIZE = (IMG_HEIGHT, IMG_WIDTH, 3)
INPUT_MASK_SIZE = (IMG_HEIGHT, IMG_WIDTH, 1)

BATCH_SIZE = 16

IMG_DIR, MASK_DIR

('./Selected/train_color', './Selected/train_label')

In [7]:
img_paths  = pd.DataFrame(dict(path = glob(os.path.join(MASK_DIR, '*.*p*g'))))
mask_paths = pd.DataFrame(dict(path = glob(os.path.join(IMG_DIR, '*.*j*g'))))

all_paths = img_paths.append(mask_paths)
all_paths['group'] = all_paths['path'].map(lambda x: x.split(f'{os.sep}')[-2].split('_')[-1])
all_paths['id'] = all_paths['path'].map(lambda x: '_'.join(os.path.splitext(os.path.basename(x))[0].split('_')[0:4]))
all_paths.head()

,path,group,id
0,./Selected/train_label/170908_061534099_Camera...,label,170908_061534099_Camera_6
1,./Selected/train_label/170908_061534516_Camera...,label,170908_061534516_Camera_5
2,./Selected/train_label/170908_061534655_Camera...,label,170908_061534655_Camera_5
3,./Selected/train_label/170908_061535350_Camera...,label,170908_061535350_Camera_5
4,./Selected/train_label/170908_061534655_Camera...,label,170908_061534655_Camera_6


In [8]:
group_df = all_paths.pivot_table(values = 'path', columns = 'group', aggfunc = 'first', index = ['id']).reset_index()
group_df.head()

group,id,color,label
0,170908_061502408_Camera_5,./Selected/train_color/170908_061502408_Camera...,./Selected/train_label/170908_061502408_Camera...
1,170908_061502408_Camera_6,./Selected/train_color/170908_061502408_Camera...,./Selected/train_label/170908_061502408_Camera...
2,170908_061502547_Camera_5,./Selected/train_color/170908_061502547_Camera...,./Selected/train_label/170908_061502547_Camera...
3,170908_061502547_Camera_6,./Selected/train_color/170908_061502547_Camera...,./Selected/train_label/170908_061502547_Camera...
4,170908_061502686_Camera_5,./Selected/train_color/170908_061502686_Camera...,./Selected/train_label/170908_061502686_Camera...


In [0]:
from sklearn.model_selection import train_test_split
train_split_df, valid_split_df = train_test_split(group_df, random_state = 2018, test_size = 0.25)

In [0]:
import cv2

def data_gen(df, size=16, normalize=True):
    if size > df.shape[0]:
        print('Size is bigger than available number of images. Please specify a smaller size than ', df.shape[0])
        return
    elif size < 0:
        print('Size should be greater than zero ', df.shape[0])

    ids = df['id'].tolist()
    random.shuffle(ids)
  
    for i in range(size):
        img = np.zeros((size, IMG_HEIGHT, IMG_WIDTH, 3)).astype('float')
        mask = np.zeros((size, IMG_HEIGHT, IMG_WIDTH, 1)).astype('float')

        img_path  = df.loc[df['id'] == ids[i]]['color'].values[0]
        mask_path = df.loc[df['id'] == ids[i]]['label'].values[0]
        #print("Pair:")
        #print("\t", img_path)
        #print("\t", mask_path)

        train_img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        if normalize:
            train_img = train_img/255.
        train_img =  cv2.resize(train_img, (IMG_WIDTH, IMG_HEIGHT))# Read an image from folder and resize

        img[i] = train_img #add to array - img[0], img[1], and so on.


        train_mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        t_mask = np.where(train_mask<33000, 0, train_mask)
        #train_mask = (t_mask//1000) + ((t_mask%1000)/100)            
        train_mask = (t_mask//1000)

        train_mask = cv2.resize(train_mask, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv2.INTER_NEAREST)
        train_mask = train_mask.reshape(IMG_HEIGHT, IMG_WIDTH, 1) # Add extra dimension for parity with train_img size [384 * 384 * 3]

        mask[i] = train_mask

        
        if((i+1)%500 == 0):
            random.shuffle(ids)
            print('randomizing again')
        
    return img, mask

In [0]:
X_train, y_train = data_gen(train_split_df, size=384, normalize=True)
X_valid, y_valid = data_gen(valid_split_df, size=32, normalize=True)

In [12]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((384, 256, 256, 3), (384, 256, 256, 1), (32, 256, 256, 3), (32, 256, 256, 1))

In [0]:
def dice_coefficient(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)

    return numerator / (denominator + tf.keras.backend.epsilon())

In [0]:
from tensorflow.keras.losses import binary_crossentropy
def loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) - tf.math.log(dice_coefficient(y_true, y_pred) + tf.keras.backend.epsilon())

In [15]:
#x_train = preprocess_input(X_train)
#x_valid = preprocess_input(X_valid)
x_train = X_train
x_valid = X_valid

model = sm.Unet(backbone_name=BACKBONE, encoder_weights='imagenet', input_shape=(None, None, 3), encoder_freeze=True)
model.summary()














Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, None, None, 3 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, None, None, 3 9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, None, None, 6 9408        zero_padding2d_1[0][0]           
_______________________________________________________________________________

In [16]:
model.compile(
    'Nadam',
    loss=loss,
    metrics=[dice_coefficient],
)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [17]:
lr_reduce = ReduceLROnPlateau(monitor='dice_coefficient', factor=0.1, epsilon=1e-5, patience=5, verbose=1)
es = EarlyStopping(monitor='dice_coefficient', patience=100, verbose=1, mode='auto')

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1335: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [18]:
model.fit(
    x=x_train,
    y=y_train,
    batch_size=32,    
    epochs=1000,
    validation_data=(x_valid, y_valid),
    callbacks=[lr_reduce, es]
)



Train on 384 samples, validate on 32 samples
Epoch 1/1000
384/384 [==============================] - 14s 38ms/step - loss: 15.7212 - dice_coefficient: 1.2941e-04 - val_loss: 33.4758 - val_dice_coefficient: 0.0000e+00
Epoch 2/1000
384/384 [==============================] - 4s 9ms/step - loss: 15.7554 - dice_coefficient: 6.9971e-06 - val_loss: 12.7368 - val_dice_coefficient: 0.0018
Epoch 3/1000
384/384 [==============================] - 4s 9ms/step - loss: 15.3625 - dice_coefficient: 3.4188e-04 - val_loss: 13.1358 - val_dice_coefficient: 0.0297
Epoch 4/1000
384/384 [==============================] - 4s 10ms/step - loss: 15.2295 - dice_coefficient: 0.0010 - val_loss: 18.6558 - val_dice_coefficient: 8.6524e-09
Epoch 5/1000
384/384 [==============================] - 4s 10ms/step - loss: 15.4409 - dice_coefficient: 5.4354e-05 - val_loss: 71.7108 - val_dice_coefficient: 0.0000e+00
Epoch 6/1000
384/384 [==============================] - 4s 10ms/step - loss: 15.3314 - dice_coefficient: 1.6273